# 第2回 (11/9)　Multi Layer Perceptron (MLP) のライブラリ無し実装
- もちろんnumpyは使います！！

## 0. MNISTデータの読み込み
- MNIST: 手書き数字の大規模データセット
    - 0〜9の10クラス
    - 訓練データ60000枚、テストデータ10000枚
    - 28\*28のグレースケール画像
- 研究室のサーバから読み込む


- X: 画像データ
    - X_trainは60000\*784, X_testは10000\*784の行列
- y: ラベル
    - np.eye(10)[〜]によってone-of-k表現にしておく
- N: サンプル数
- 一般的なDeep Learningライブラリでは変数はfloat32もしくはint32型に統一するのでここでもそうしておく

In [ ]:
import numpy as np

dir = '/data/ishimochi0/dataset/mnist/'

#  訓練データ
X_train = np.loadtxt(dir + 'train-images.txt').reshape((-1, 784)).astype(np.float32) / 255.
y_train = np.loadtxt(dir + 'train-labels.txt').astype(np.int32)
y_train = np.eye(10)[y_train].astype(np.int32)
N_train = len(X_train)

# テストデータ
X_test = np.loadtxt(dir + 'test-images.txt').reshape((-1, 784)).astype(np.float32) / 255.
y_test = np.loadtxt(dir + 'test-labels.txt').astype(np.int32)
y_test = np.eye(10)[y_test].astype(np.int32)
N_test = len(X_test)

## 1. 活性化関数とその微分

### sigmoid関数
$$
\sigma(x) = \frac{1} {1+e^{-x}}
$$
<img src="figure/sigmoid.png", width=300>

In [ ]:
class Sigmoid:
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))# WRITE ME!
    
    def deriv(self, x):
        return self(x) * (1 -  self(x))# WRITE ME!

#### 答え合わせ

In [ ]:
sigmoid = Sigmoid()

x = np.arange(10).reshape((2, 5)) - 5
print(x)

print(sigmoid(x)) # call関数(=普通のsigmoid関数)が実行される
print(sigmoid.deriv(x)) # 導関数が実行される

### relu関数
$$
relu(x) = max(0, x)
$$
<img src="figure/relu.png", width=300>

#### ヒント
- 入力xはデータ数\*次元数の行列であることに注意
- 「x>0」とすると、xの各成分の正負に応じたTrueとFalseからなる行列が得られる
- Trueは1, Falseは0として振舞う

In [ ]:
x = np.arange(10).reshape((2, 5)) - 5
print(x)

print(x > 0)

print(1 * True)
print(1 * False)

In [ ]:
class ReLU:
    def __call__(self, x):
        return x * (x > 0)# WRITE ME!
    
    def deriv(self, x):
        return 1 * (x > 0)# WRITE ME!

#### 答え合わせ

In [ ]:
relu = ReLU()

x = np.arange(10).reshape(2, 5) - 5
print(x)

print(relu(x))
print(relu.deriv(x))

### softmax関数
$$
softmax(x_{k}) = \frac{e^{x_{k}}} {\Sigma_{i=1}^{K}{e^{x_{i}}}}
$$
- 多クラス識別ネットワークの出力層に必ず使用する活性化関数
- xの各成分の対数を取り、各行の和が1になるよう正規化する
- 各データが各クラスに属する確率を表す

#### ヒント
各行の和を取るにはnp.sum()を使うが...
- 普通にnp.sum(a)とすると、aの全要素の和が計算されてしまう。行方向の和を取るには??
- それだけだと、割り算のサイズが合わないと怒られてしまう。行方向の和を取ったときに、行方向の次元数が1になってしまうからである。それを回避するには??
 - Shift+tabで関数の使い方を見てみよう

In [ ]:
class Softmax:
    def __call__(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True) # WRITE ME! 

    def deriv(self, x):
        return self(x) * (1 -  self(x))# WRITE ME!

#### 答え合わせ

In [ ]:
softmax = Softmax()

x = np.arange(10).reshape(2, 5) - 5
print(x)

print(softmax(x))
print(softmax.deriv(x))

## 2. 線形層クラス
- 作成時の引数: 入力次元数in_dim、出力次元数out_dim、活性化関数の種類


- 内部パラメータ: 重みW(=in_dim*out_dim), バイアスb(=out_dim), 誤差delta
 - Wの初期値は一様分布に従う乱数。範囲は上手い決め方があるが今回は-0.08〜0.08とする
 - bの初期値は0ベクトル
 - どちらもfloat32型にしておこう
 
 
- call関数: 順伝播の計算
 - u: 入力xに重みWとバイアスbを適用した結果
 - z: uに活性化関数を適用した結果

In [ ]:
class Linear:
    def __init__(self, in_dim, out_dim, activation):
        self.W = np.random.uniform(low=-0.08, high=0.08, size=(in_dim, out_dim)).astype(np.float32)# WRITE ME!
        self.b = np.zeros(out_dim).astype(np.float32)# WRITE ME!
        self.delta = None
        self.activation = activation()

    def __call__(self, x):
        self.u = np.dot(x, self.W) + self.b# WRITE ME!
        self.z = self.activation(self.u)# WRITE ME!
        return self.z

## 3. MLPクラス
- 作成時の引数: 層のリストlayers=[Linear(〜), Linear(〜), ...]
- 3層の場合の例:
<img src="figure/train.png", width=800>

In [ ]:
class MLP():
    def __init__(self, layers):
        self.layers = layers
        
    def train(self, x, t, lr):     
        # 1. 順伝播
        self.y = x
        for layer in self.layers:
            self.y = layer(self.y)# WRITE ME!
        self.loss = np.sum(-t*np.log(self.y)) / len(x)# WRITE ME!
        
        # 2. 誤差逆伝播
        # 最終層の誤差
        delta = self.y - t# WRITE ME!
        self.layers[-1].delta = delta
        W = self.layers[-1].W
        
        # 中間層の誤差を計算
        # 各ループ開始時に、一つ上の層の誤差と重みがそれぞれdelta、Wに格納されている
        for layer in self.layers[-2::-1]:
            delta = np.dot(delta, W.T) * layer.activation.deriv(layer.u)# WRITE ME!
            layer.delta = delta
            W = layer.W
        
        # 3. 各層のパラメータを更新
        # 各ループ開始時に、一つ下の層の出力がzに格納されている
        z = x
        for layer in self.layers:
            dW = np.dot(z.T, layer.delta)# WRITE ME!
            db = np.dot(np.ones(len(z)), layer.delta)# WRITE ME!
            layer.W -= lr * dW# WRITE ME!
            layer.b -= lr * db# WRITE ME!
            z = layer.z
            
        return self.loss
    
    
    def test(self, x, t):
        # 順伝播 (train関数と全く同じでOK)
        self.y = x
        for layer in self.layers:
            self.y = layer(self.y)# WRITE ME!
        self.loss = np.sum(-t*np.log(self.y)) / len(x)# WRITE ME!
        return self.loss

- 準備はここまで。

## 4. モデルの定義

In [ ]:
model = MLP([Linear(784, 1000, Sigmoid),
                        Linear(1000, 1000, Sigmoid),
                        Linear(1000, 10, Softmax)])

## 5. MNISTの学習
- 来週chainerを使って実装するときとほっとんど同じ書き方をしました！
- 注意点
 - 学習時は毎回順番がランダムになるようにする
 - accuracyの計算
 - print文はカンマを付けると改行されない
 - 学習に影響はないが、時々log0エラーが出てlossがnanになることがある

In [ ]:
n_epoch = 20
batchsize = 100
lr = 0.01

for epoch in range(n_epoch):
    print('epoch %d |' % epoch,)
    
    # Training
    sum_loss = 0
    pred_y = []
    perm = np.random.permutation(N_train)
    
    for i in range(0, N_train, batchsize):
        x = X_train[perm[i:i+batchsize]]
        t = y_train[perm[i:i+batchsize]]
        
        sum_loss += model.train(x, t, lr) * len(x)
        pred_y.extend(np.argmax(model.y, axis=1))
    
    loss = sum_loss / N_train
    accuracy = np.sum(np.eye(10)[pred_y] * y_train[perm]) / N_train
    print('Train loss %.3f, accuracy %.4f |' %(loss, accuracy),)
    
    
    # Testing
    sum_loss = 0
    pred_y = []
    
    for i in range(0, N_test, batchsize):
        x = X_test[i: i+batchsize]
        t = y_test[i: i+batchsize]
        
        sum_loss += model.test(x, t) * len(x)
        pred_y.extend(np.argmax(model.y, axis=1))

    loss = sum_loss / N_test
    accuracy = np.sum(np.eye(10)[pred_y] * y_test) / N_test
    print('Test loss %.3f, accuracy %.4f' %(loss, accuracy))

- デフォルトのネットワークでは、97%台後半程度の識別精度が出たと思います。
- 中間層の活性化関数をReLUに変えてみたり、層数を増やしてみたりすると...?